# SDGSAT-1 cloud fill preprocessing code

In [ ]:
import os
import arcpy
import glob
import numpy as np
from arcpy.sa import *

def RemoveDir(filepath):
    filelist = glob.glob(filepath+"\\*.tif")+ glob.glob(filepath+"\\*.shp")
    for f in filelist:
        filepath_t = os.path.join(filepath, f)
        if os.path.isfile(filepath_t):
            arcpy.Delete_management(filepath_t)
    os.rmdir(filepath)

def MakeDir(work_path,name):
    if not os.path.exists(os.path.join(work_path,name)):
        os.makedirs(os.path.join(work_path,name))
    temp_path = os.path.join(work_path,name)
    return temp_path

# --------------------
work_path = r"Path"
Cloud_Name = ""
Target_date = ""

# First, noise removal

Method from《Efficacy of the SDGSAT-1 glimmer imagery in measuring sustainable development goal indicators 7.1.1, 11.5.2, and target 7.3》

The source code is located at Matlab Code -> Denoise which should be run in Matlab.

# Second, image mosaicing 

In [ ]:
raw_file_path = os.path.join(work_path,"R2_Denoise_Selected_NTL")
arcpy.env.workspace = raw_file_path
arcpy.env.nodata = "None"
rasters = arcpy.ListRasters("*", "tif")

mosaic_path = MakeDir(work_path,"R3_Mosaic_Selected_NTL")
date_list = []
for raster in rasters:
    arcpy.management.SetRasterProperties(raw_file_path+ '\\'+raster, nodata="1 0;2 0;3 0")
    date_list.append(raster[9:17])

arr = np.array(date_list)
unique, counts = np.unique(arr, return_counts=True)
repeated_images = unique[np.where(counts > 1)[0]]

if len(repeated_images) >= 1:

    print("Dates",repeated_images,"need to be mosaicked")

    for raster in rasters:
        if any(repeated_image in raster for repeated_image in repeated_images):
            continue
        else:
            if os.path.exists(mosaic_path+ '\\'+raster[0:-10]+".tif"):
                continue
            else:
                arcpy.management.CopyRaster(raster, mosaic_path+ '\\'+raster[0:-10]+".tif")
            
    for repeated_image in repeated_images:
        temp_mosaic = []
        for raster in rasters:
            if repeated_image in raster:
                temp_mosaic.append(raster)
        arcpy.MosaicToNewRaster_management(temp_mosaic,mosaic_path,temp_mosaic[0][0:-10]+".tif","","16_BIT_UNSIGNED", "40", "3", "MEAN","MATCH")
        print(repeated_image,"is Done")
else: 
    for raster in rasters:
        arcpy.management.CopyRaster(raster, mosaic_path+ '\\'+raster[0:-10]+".tif")

# Third, reprojection

In [ ]:
mosaic_path = os.path.join(work_path,"R3_Mosaic_Selected_NTL")
arcpy.env.workspace = mosaic_path
arcpy.env.nodata = "None"

rasters = arcpy.ListRasters("*", "tif")
re_file_path = MakeDir(work_path,"R4_Reproject_Selected_NTL")

proj_list = []
for raster in rasters:
    currentRaster = Raster(raster)
    current_spatial_reference = arcpy.Describe(currentRaster).spatialReference
    proj_list.append(current_spatial_reference.name)

max_proj = max(proj_list,key=proj_list.count)


if len(proj_list) == proj_list.count(max_proj):
    for raster in rasters:
        arcpy.management.CopyRaster(raster, re_file_path+ '\\'+raster)

else:
    print("Projection is not consistent")
    Target_spatial_reference = arcpy.Describe(Raster(rasters[proj_list.index(max_proj)])).spatialReference

    for raster in rasters:
        currentRaster = Raster(raster)
        current_spatial_reference_name = arcpy.Describe(currentRaster).spatialReference.name

        if current_spatial_reference_name != max_proj:
            arcpy.ProjectRaster_management(currentRaster,
                                           re_file_path+ '\\'+raster,
                                           Target_spatial_reference,
                                           "BILINEAR",
                                           "40")
        else:
            arcpy.management.CopyRaster(raster, re_file_path+ '\\'+raster)
# --------------------

print("Done")

# Fourth, image registration

In [ ]:
import os
import shutil
from arosics import COREG_LOCAL

work_path = r'Path'
Target_date = ""

if not os.path.exists(os.path.join(work_path,"R5_Registration_Selected_NTL")):
    os.makedirs(os.path.join(work_path,"R5_Registration_Selected_NTL"))
ref_path = os.path.join(work_path,"R5_Registration_Selected_NTL")

re_file_path = os.path.join(work_path,"R4_Reproject_Selected_NTL")
NTL_images = os.listdir(re_file_path)

ref_image = ''
tar_images = []

for NTL_image in NTL_images:
    if Target_date in NTL_image and NTL_image[-3:] == "tif":
        ref_image = NTL_image
    elif NTL_image[-3:] == "tif":
        tar_images.append(NTL_image)
    else:
        pass

for tar_image in tar_images:

    im_reference = ref_image
    im_target    = tar_image

    kwargs = {
        'grid_res'     : 40,
        'max_points'   : 100000,
        'path_out'     : ref_path+"\\"+im_target,
        'fmt_out'      : 'GTIFF',
        'CPUs'         : 12
    }

    CRL = COREG_LOCAL(re_file_path+"\\"+im_reference,re_file_path+"\\"+im_target,**kwargs)
    CRL.correct_shifts()

shutil.copyfile(re_file_path+"\\"+im_reference, ref_path+"\\"+im_reference)

# Fifth, maximum common subregion extraction

In [ ]:
ref_path = os.path.join(work_path,"R5_Registration_Selected_NTL")
arcpy.env.workspace = ref_path
temp_rasters = arcpy.ListRasters("*", "tif")

extent_path = MakeDir(work_path,"R6_Selected_Extent")

for temp_raster in temp_rasters:
    key_names = temp_raster.split("_")
    temp_raster_band_0 = arcpy.ia.ExtractBand(temp_raster, 1)
    extent_raster = RasterCalculator([temp_raster_band_0], ["x"],"SetNull(IsNull(x),1)", "FirstOf", "FirstOf")
    extent_raster_Int = Int(extent_raster)
    arcpy.conversion.RasterToPolygon(extent_raster_Int, extent_path+ '\\'+key_names[2]+key_names[5]+ "_Extent.shp", "NO_SIMPLIFY")


arcpy.env.workspace = extent_path
list_shapefiles = arcpy.ListFiles("*.shp")
arcpy.analysis.Intersect(list_shapefiles,"Max_Extent.shp")

print("Done")

# Sixth, data snap and clip

In [ ]:
extent_path = os.path.join(work_path,'R6_Selected_Extent')
arcpy.env.workspace = ref_path
arcpy.env.nodata = "None"
temp_rasters = arcpy.ListRasters("*", "tif")

Snap_Clip_path = MakeDir(work_path,"R7_Snap_Clip_Selected_NTL")
outExtractByMask = ExtractByMask(temp_rasters[0], extent_path+'\\'+"Max_Extent.shp", "INSIDE")
outExtractByMask_0 = RasterCalculator([outExtractByMask], ["x"],"Con(IsNull(x),0,x)", "FirstOf", "FirstOf")
outExtractByMask_0.save(Snap_Clip_path+'\\'+temp_rasters[0])
arcpy.management.SetRasterProperties(Snap_Clip_path+'\\'+temp_rasters[0], nodata="1 0;2 0;3 0")

arcpy.env.extent = Raster(Snap_Clip_path+'\\'+temp_rasters[0]).extent
arcpy.env.snapRaster = Raster(Snap_Clip_path+'\\'+temp_rasters[0])

raster_num = 0
for temp_raster in temp_rasters:

    if raster_num == 0:
        raster_num += 1
        continue

    outExtractByMask = ExtractByMask(temp_raster, extent_path+'\\'+"Max_Extent.shp", "INSIDE")
    outExtractByMask_0 = RasterCalculator([outExtractByMask], ["x"],"Con(IsNull(x),0,x)", "FirstOf", "FirstOf")
    outExtractByMask_0.save(Snap_Clip_path+'\\'+temp_raster)
    arcpy.management.SetRasterProperties(Snap_Clip_path+'\\'+temp_raster, nodata="1 0;2 0;3 0")
    raster_num += 1
    
print("Done")

# Seventh, cloud masking

In [ ]:
Snap_Clip_path = os.path.join(work_path,"NTL")
arcpy.env.workspace = Snap_Clip_path
arcpy.env.nodata = "None"
temp_rasters = arcpy.ListRasters("*", "tif")
arcpy.env.snapRaster = temp_rasters[0]
arcpy.env.extent = temp_rasters[0]

temp_path = MakeDir(work_path,"temp")

arcpy.conversion.FeatureToRaster(work_path+'\\'+'Cloud'+'\\'+Cloud_Name+'.shp',"FID",temp_path+'\\'+"temp_cloud.tif",40)
cloud_ras_cal = RasterCalculator([temp_path+'\\'+"temp_cloud.tif"], ["x"],"Con(x>=0,1,0)", "FirstOf", "FirstOf")
cloud_ras_cal.save(temp_path+'\\'+"temp_cloud_1.tif")
arcpy.management.SetRasterProperties(temp_path+'\\'+"temp_cloud_1.tif", nodata="1 0")
cloud_ras_cal_1 = RasterCalculator([temp_path+'\\'+"temp_cloud_1.tif"], ["x"],"Con(x==1,1,0)", "FirstOf", "FirstOf")
cloud_ras_cal_1.save(work_path+'\\'+'Cloud'+'\\'+'Cloud.tif')
arcpy.Delete_management(temp_path+'\\'+"temp_cloud.tif")
arcpy.Delete_management(temp_path+'\\'+"temp_cloud_1.tif")

Target_name = []
for temp_raster in temp_rasters:
    
    if Target_date in temp_raster:
        
        Target_name = temp_raster
        for band in range(3):
            temp_raster_band = arcpy.ia.ExtractBand(temp_raster, band+1)
            ExtractByCloud = RasterCalculator([temp_raster_band,work_path+'\\'+'Cloud'+'\\'+"Cloud.tif"], ["x","y"],"Con(y==1,0,Con(IsNull(x),0,x))", "FirstOf", "FirstOf")
            ExtractByCloud.save(temp_path+'\\'+str(band)+".tif")
            arcpy.management.SetRasterProperties(temp_path+'\\'+str(band)+".tif", nodata="1 0")


arcpy.env.workspace = temp_path
arcpy.env.nodata = "None"
temp_rasters = arcpy.ListRasters("*", "tif")
REAL_path = MakeDir(work_path,"REAL")
arcpy.management.CopyRaster(Snap_Clip_path+'\\'+Target_name, REAL_path+ '\\'+Target_name)
arcpy.Delete_management(Snap_Clip_path+'\\'+Target_name)
arcpy.CompositeBands_management(temp_rasters,Snap_Clip_path+'\\'+Target_name)
arcpy.management.SetRasterProperties(Snap_Clip_path+'\\'+Target_name, nodata="1 0;2 0;3 0")

print("Done")
RemoveDir(temp_path)

# Eighth, mean synthesis

In [ ]:
Snap_Clip_path = os.path.join(work_path,"NTL")
arcpy.env.workspace = Snap_Clip_path
arcpy.env.nodata = "None"

output_folder = MakeDir(work_path,"Annual")

output_raster_name = "YearMEAN"
input_rasters = arcpy.ListRasters()
arcpy.env.snapRaster = input_rasters[0]

raster_list1 = []
raster_list2 = []

for raster in input_rasters:
    raster_path = arcpy.env.workspace + "/" + raster
    raster_list1.append(raster_path)

    raster01 = arcpy.sa.Con(IsNull(raster), 0, 1)
    raster_list2.append(raster01)

arcpy.env.extent = "MAXOF"
count_raster = CellStatistics(raster_list2, "SUM", "DATA", "MULTI_BAND")

output = arcpy.sa.Con(count_raster > 1, CellStatistics(raster_list1, "MEAN", "DATA", "MULTI_BAND"), )
output_0 = RasterCalculator([output], ["x"],"Con(IsNull(x),0,x)", "FirstOf", "FirstOf")
output_0.save(output_folder + "/" + output_raster_name + ".tif")
arcpy.management.SetRasterProperties(output_folder + "/" + output_raster_name + ".tif", nodata="1 0;2 0;3 0")

print("Done")

# Ninth, calculation of SONDI index

In [ ]:
arcpy.env.workspace = Snap_Clip_path
arcpy.env.nodata = "None"
temp_rasters = arcpy.ListRasters("*", "tif")

Input_Spectrum_path = MakeDir(work_path,"Input_Spectrum")
Target_Spectrum_path = MakeDir(work_path,"Target_Spectrum")

for temp_raster in temp_rasters:
    raster_band_r = arcpy.ia.ExtractBand(temp_raster, 1)
    raster_band_g = arcpy.ia.ExtractBand(temp_raster, 2)
    raster_band_b = arcpy.ia.ExtractBand(temp_raster, 3)

    NDIGR = (raster_band_g - raster_band_r)/(raster_band_g + raster_band_r)
    NDIBG = (raster_band_b - raster_band_g)/(raster_band_b + raster_band_g)

    SONDI = NDIGR + NDIBG

    if Target_date in temp_raster:
        output_0 = RasterCalculator([SONDI], ["x"],"Con(IsNull(x),3,x)", "FirstOf", "FirstOf")
        output_0.save(Target_Spectrum_path+'\\'+temp_raster)
        arcpy.management.SetRasterProperties(Target_Spectrum_path+'\\'+temp_raster, nodata="1 3")
    else:
        output_0 = RasterCalculator([SONDI], ["x"],"Con(IsNull(x),3,x)", "FirstOf", "FirstOf")
        output_0.save(Input_Spectrum_path+'\\'+temp_raster)
        arcpy.management.SetRasterProperties(Input_Spectrum_path+'\\'+temp_raster, nodata="1 3")

print("Done")


# Tenth, copy related files to the specified path

In [ ]:
Snap_Clip_path = os.path.join(work_path,"NTL")

arcpy.env.workspace = Snap_Clip_path
arcpy.env.nodata = "None"
temp_rasters = arcpy.ListRasters("*", "tif")

Input_path = MakeDir(work_path,"Input")
Target_path = MakeDir(work_path,"Target")
for temp_raster in temp_rasters:
    if Target_date in temp_raster:
        arcpy.management.CopyRaster(temp_raster,Target_path+ '\\'+temp_raster)
    else:
        arcpy.management.CopyRaster(temp_raster,Input_path+ '\\'+temp_raster)
        
print("Done!")